# <h1 align="center"><font color="red">Sistemas de Recomendações com Surprise</font></h1>

<font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

# <font color="gree">Contextualizando</font>

<font color="orange">[Surprise é um scikit](https://surpriselib.com/) Python para construir e analisar `Sistemas de Recomendação` que lidam com dados de `classificação explícitos`. 

Mas ainda não sabe do que estou falando? Usa o YouTube? Ou Netflix? Ou qualquer plataforma de Streaming? Portanto, todas elas usam `sistemas de recomendação` para sugerir o melhor conteúdo para você.</font>

# <font color="gree">Sistemas de Recomendações</font>

<font color="orange">Os sistemas de recomendações nada mais são que algoritmos com diversas técnicas computacionais combinadas para `entregar ao usuário itens personalizados`. Sendo assim, usando o exemplo do YouTube, à medida que você assiste conteúdos na plataforma ele sugere novos conteúdos similares. Assim, por debaixo dos panos, um algoritmo de Inteligência Artificial combina seu perfil, com outros usuários e conteúdos similares, para te sugerir `“o que você quer assistir e nem sabia ainda”.`

Afinal, será que o Python também tem uma ferramenta para facilitar a construção de sistemas de recomendações? E a resposta é: sim ou claro! Então vamos conhecer a `biblioteca Surprise`.</font>

# <font color="gree">O Surprise</font>

Primeiramente vamos na documentação, onde a biblioteca está definida (em tradução livre) da seguinte forma:

“Surprise é um pacote Python para construir e analisar sistemas de recomendação que lidam com dados de `classificação explícitos`.”


Ainda de acordo com a documentação da biblioteca que é bem completa, com a proposta de dar autonomia ao desenvolvedor. O pacote `Surprise` já disponibiliza diversos algoritmos prontos para previsão (`vamos utilizar 3 deles aqui`). Além disso possui ferramentas para avaliar os modelos.

Sendo assim, podemos resolver problemas de recomendações com a `biblioteca surprise`.

# <font color="gree">O Problema de Recomendação</font>

<font color="orange">Nosso problema é como recomendar filmes para usuários de uma plataforma, `The Filmes Top`. Na `The Filmes Top`, o usuário pode avaliar os filmes com uma nota de `0 a 5`, qualquer semelhança com outras plataformas é mera coincidência. Portanto, <font color="red">como sugerir novos filmes para o usuário baseado nas suas experiências e dos outros usuários?</font>

Logo, para resolver nosso problema, vamos usar o dataset do [MovieLens Latest Datasets](https://grouplens.org/datasets/movielens/), com 100 mil avaliações de 600 usuários em 9 mil filmes.

A recomendação de filmes é algo comum encontrado nas plataformas de streaming, mas os sistemas de recomendações no geral estão presentes em muitas outras aplicações.</font>


# <font color="pink">Instalação e Importando Bibliotecas</font>

Instalação:
```
pip install surprise
```

In [1]:
import pandas as pd
from surprise import Reader, Dataset
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise import accuracy

# <font color="pink">Datasets</font>

In [2]:
# Dataset com o detalhe dos Filmes:
movies = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/movies.csv')

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies.shape

(9742, 3)

In [4]:
# Dataset com as avaliações:
ratings = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/ratings.csv')

ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
ratings.shape

(100836, 4)

# <font color="gree">Visualizando o Dataset</font>

In [6]:
# Juntando as informações de filmes e avaliações:
filmes = ratings.join(movies.set_index('movieId'), on='movieId')

filmes.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [7]:
filmes.shape

(100836, 6)

In [8]:
filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
 4   title      100836 non-null  object 
 5   genres     100836 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 4.6+ MB


In [9]:
# Números dos datasets:
print('Quantidade de Filmes Avaliados: ',
filmes['movieId'].value_counts().shape[0])
 
print('Quantidade de Usuários Avaliando: ',
filmes['userId'].value_counts().shape[0])
 
print('Quantidade de Avaliações: ', ratings.shape[0])

Quantidade de Filmes Avaliados:  9724
Quantidade de Usuários Avaliando:  610
Quantidade de Avaliações:  100836


<font color="orange">Quais são os `TOP5` filmes com avaliações?</font>

In [10]:
# Quantidade de Avaliações TOP5 Filmes:
filmes['title'].value_counts().head()

title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: count, dtype: int64

<font color="orange">Qual a quantidade de avaliações por usuário?</font>

In [11]:
# Quantidade de Avaliações TOP5 Usuários:
filmes['userId'].value_counts().head()


userId
414    2698
599    2478
474    2108
448    1864
274    1346
Name: count, dtype: int64

<font color="orange">Quais são os filmes avaliados por um determinado usuário?</font>

In [12]:
# Avaliações do usuário 414
filmes.query('userId == 414').head()

,userId,movieId,rating,timestamp,title,genres
62294,414,1,4.0,961438127,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
62295,414,2,3.0,961594981,Jumanji (1995),Adventure|Children|Fantasy
62296,414,3,4.0,961439278,Grumpier Old Men (1995),Comedy|Romance
62297,414,5,2.0,961437647,Father of the Bride Part II (1995),Comedy
62298,414,6,3.0,961515642,Heat (1995),Action|Crime|Thriller


In [15]:
# Avaliações do usuário 414
filmes.query('userId == 1').head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
